In [1]:
import os
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [2]:
TRAIN_DIR = 'data/train'
TEST_DIR = 'data/test'

In [3]:
train_datagen = ImageDataGenerator( 
    rescale=1./255,
    rotation_range = 30,
    horizontal_flip = True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.3,
    zoom_range=0.3,
    fill_mode = 'nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

In [4]:
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    color_mode='grayscale',
    target_size=(48,48),
    batch_size=32,
    class_mode='categorical',
    shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
    TEST_DIR,
    color_mode='grayscale',
    target_size=(48,48),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [5]:
class_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

img, label = train_generator.__next__()

In [9]:
num_classes = 7

model = Sequential()

# Block 1
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(48, 48, 1)))

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

# Block 2
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))


# Block 3
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))


# Reduce learning rate if validation loss plateaus
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

# Early stopping to prevent overfitting
early_stop = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001, decay=1e-6), metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 48, 48, 32)        320       
                                                                 
 conv2d_5 (Conv2D)           (None, 48, 48, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 24, 24, 64)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 24, 24, 64)        0         
                                                                 
 conv2d_6 (Conv2D)           (None, 24, 24, 128)       73856     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 12, 12, 128)      0         
 2D)                                                  

In [10]:
no_of_train_images = 0
for rrot, dirs, files in os.walk(TRAIN_DIR):
    no_of_train_images += len(files)
    
    
no_of_test_images = 0
for rrot, dirs, files in os.walk(TEST_DIR):
    no_of_test_images += len(files)
    

In [11]:

model.fit(train_generator,
          steps_per_epoch=no_of_train_images//32,
          epochs=150,
          validation_data=validation_generator,
          validation_steps=no_of_test_images//32,
          callbacks=[reduce_lr, early_stop])

Epoch 1/150
897/897 [==============================] - 23s 25ms/step - loss: 1.7988 - accuracy: 0.2543 - val_loss: 1.7507 - val_accuracy: 0.2866 - lr: 1.0000e-04
Epoch 2/150
897/897 [==============================] - 22s 25ms/step - loss: 1.7334 - accuracy: 0.2949 - val_loss: 1.6265 - val_accuracy: 0.3662 - lr: 1.0000e-04
Epoch 3/150
897/897 [==============================] - 23s 25ms/step - loss: 1.6755 - accuracy: 0.3353 - val_loss: 1.5699 - val_accuracy: 0.3708 - lr: 1.0000e-04
Epoch 4/150
897/897 [==============================] - 23s 25ms/step - loss: 1.6096 - accuracy: 0.3677 - val_loss: 1.4316 - val_accuracy: 0.4584 - lr: 1.0000e-04
Epoch 5/150
897/897 [==============================] - 22s 25ms/step - loss: 1.5486 - accuracy: 0.4015 - val_loss: 1.4096 - val_accuracy: 0.4576 - lr: 1.0000e-04
Epoch 6/150
897/897 [==============================] - 22s 25ms/step - loss: 1.4927 - accuracy: 0.4193 - val_loss: 1.3239 - val_accuracy: 0.5054 - lr: 1.0000e-04
Epoch 7/150
897/897 [=======

In [12]:
model.save("model_weights.h5")